In [100]:
import requests
import json
import csv
import time

In [101]:
def fetch_item_price(item_id):
    item_data = data.get("data", {}).get(str(item_id), {})

    if not item_data:
        print(f"Item ID {item_id} not found in the API response.")
        return None

    return {
        "id": item_id,
        "high": item_data.get("avgHighPrice"),
        "low": item_data.get("avgLowPrice")
    }


In [102]:
def save_to_csv(data, filename="osrs_prices.csv"):
    """
    Appends data to a CSV file.

    Args:
        data (list): List of dictionaries containing item data.
        filename (str): The CSV file to write to.
    """
    file_exists = False
    try:
        # Check if the file already exists
        with open(filename, "r") as file:
            file_exists = True
    except FileNotFoundError:
        pass

    with open(filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Item ID", "High Price", "Low Price"])  # Header row if file doesn't exist
        for item in data:
            writer.writerow([item["id"], item["high"], item["low"]])
    print(f"Data appended to {filename}")

In [110]:
 url = "https://prices.runescape.wiki/api/v1/osrs/1h"

response = requests.get(url, headers={
'User-Agent': '@el on discord'
})
response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

data = response.json()


all_data = []
print("Old School RuneScape Grand Exchange Price Scraper")

# Example: List of item IDs to fetch
item_ids = [4151, 11840, 12934]  # Abyssal whip, Dragon boots, Zulrah's scales

for item_id in item_ids:
    item_data = fetch_item_price(item_id)

    if item_data:
        all_data.append(item_data)
        print(f"Item ID: {item_data['id']} - High Price: {item_data['high']} - Low Price: {item_data['low']}")
    else:
        print(f"Failed to fetch data for item ID {item_id}")
        

save_to_csv(all_data)

Old School RuneScape Grand Exchange Price Scraper
Item ID: 4151 - High Price: 1411663 - Low Price: 1395905
Item ID: 11840 - High Price: 119276 - Low Price: 116272
Item ID: 12934 - High Price: 291 - Low Price: 287
Data appended to osrs_prices.csv
